In [122]:
import os, sys
currentdir = os.path.dirname(os.path.realpath("Tutorial_Rocks_Inversion.ipynb"))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

In [137]:
# MODULES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from modules import sequences, minerals, fluids
from modules.sequences import SedimentaryBasin
from modules import geophysics
from pandas.plotting import scatter_matrix
import seaborn as sn
from scipy.optimize import nnls, minimize
import scipy.optimize as opt

In [124]:
# Test feldspathic sandstone generation within SedimentaryBasin class
data = SedimentaryBasin()
data_sandstone = data.create_sandstone(thickness=10, keyword="feldspathic")

data = sequences.DataProcessing(dataset=data_sandstone)
density_bulk = data.extract_densities(type="rock", keyword="bulk")
vP = data.extract_seismic_velocities(type="rock", keyword="vP")
vS = data.extract_seismic_velocities(type="rock", keyword="vS")
gr = data.extract_gamma_ray(type="rock")
pe = data.extract_photoelectricity(type="rock")

d = []
for i in range(len(data_sandstone)):
    d.append([density_bulk[i]*1000, vP[i], vS[i], gr[i], pe[i]])
    print("Sample", i+1, ":", d[i])
d = np.array(d)

Sample 1 : [2120.0, 2799.17, 1550.48, 51.399, 1.851]
Sample 2 : [2100.0, 2754.81, 1513.01, 72.222, 1.969]
Sample 3 : [2155.0, 2845.58, 1597.05, 3.81, 1.663]
Sample 4 : [2167.0, 2871.25, 1620.89, 31.504, 1.755]
Sample 5 : [2127.0, 2877.77, 1585.48, 4.688, 1.64]
Sample 6 : [2065.0, 2785.34, 1488.84, 48.207, 1.921]
Sample 7 : [2157.0, 2896.2, 1608.3, 23.881, 1.906]
Sample 8 : [2108.0, 2845.79, 1548.09, 1.553, 1.743]
Sample 9 : [2060.0, 2766.08, 1482.31, 33.28, 1.763]
Sample 10 : [2072.0, 2765.37, 1484.81, 3.833, 1.748]


In [125]:
data_qz = minerals.oxides.quartz("")
data_kfs = minerals.feldspars().alkalifeldspar(keyword="Kfs")
data_pl = minerals.feldspars().plagioclase(keyword="Pl")
water = fluids.Water.water("")

properties_rho = [data_qz[2], data_kfs[2], data_pl[2], water[2]]
properties_vP = [data_qz[4][0], data_kfs[4][0], data_pl[4][0], water[4][0]]
properties_vS = [data_qz[4][1], data_kfs[4][1], data_pl[4][1], water[4][1]]
properties_GR = [data_qz[5][0], data_kfs[5][0], data_pl[5][0], water[5][0]]
properties_PE = [data_qz[5][1], data_kfs[5][1], data_pl[5][1], water[5][1]]

G = np.array([properties_rho, properties_vP, properties_vS, properties_GR, properties_PE])
print(G)

G_T = np.transpose(G)
G_TG = G_T@G
GTG_inv = np.linalg.inv(G_TG)
m = GTG_inv@G_T@d[0]
print(m)

[[2648.60 2596.70 2710.80 1000.00]
 [5753.20 8441.60 8750.50 1449.10]
 [4075.90 5030.30 5162.90 0.00]
 [0.00 102.21 0.00 0.00]
 [1.65 2.04 2.46 0.36]]
[0.90 0.50 -0.90 0.87]


In [131]:
m = np.linalg.lstsq(a=G, b=d[0], rcond=None)[0]
print("Model estimation:", m, m.sum())
print("Check:")
print("Density:", round(np.dot(m,properties_rho), 2))
print("vP:", round(np.dot(m,properties_vP), 2))
print("vS:", round(np.dot(m,properties_vS), 2))
print("GR:", round(np.dot(m,properties_GR), 2))
print("PE:", round(np.dot(m,properties_PE), 2))
print("Reality:")
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
print(d[0])

Model estimation: [0.90 0.50 -0.90 0.87] 1.3719723429375024
Check:
Density: 2120.0
vP: 2799.17
vS: 1550.48
GR: 51.39
PE: 0.61
Reality:
[2120.00 2799.17 1550.48 51.40 1.85]


In [146]:
A = G
b = d[0]

m0, rnorm = nnls(A, b)
print("Model estimation:", m0, m0.sum())
print("Check:")
print("Density:", round(np.dot(m0,properties_rho), 2))
print("vP:", round(np.dot(m0,properties_vP), 2))
print("vS:", round(np.dot(m0,properties_vS), 2))
print("GR:", round(np.dot(m0,properties_GR), 2))
print("PE:", round(np.dot(m0,properties_PE), 2))
print("Reality:")
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
print(d[0])

m = m0
def fn(m, A, b):
    return np.linalg.norm(A.dot(m) - b)

cons = {'type': 'eq', 'fun': lambda m:  np.sum(m)-1}
bounds = [[0., 1],[0., 1],[0., 1], [0., 1]]

minout = minimize(fn, m0, args=(A, b), method='SLSQP', bounds=bounds, constraints=cons)
m = minout.m

print(m, m.sum(), fn(m, A, b))

Model estimation: [0.35 0.00 0.00 0.77] 1.1131684897948686
Check:
Density: 1682.99
vP: 3100.75
vS: 1408.78
GR: 0.0
PE: 0.85
Reality:
[2120.00 2799.17 1550.48 51.40 1.85]


AttributeError: m

In [192]:
import numpy as np
from scipy.optimize import minimize 
from scipy.optimize import nnls 

#Define problem
A = np.array([[60, 90, 120], 
              [30, 120, 90]])
A = G

b = np.array([67.5, 60])
b = d[0]

#Use nnls to get initial guess
x0, rnorm = nnls(A, b)

#Define minimisation function
def fn(x, A, b):
    return np.linalg.norm(A.dot(x) - b)

#Define constraints and bounds
cons = {'type': 'eq', 'fun': lambda x:  np.sum(x)-1}
bounds = [[0.5, 1.0], [0., 0.5], [0., 0.5], [0., 0.2]]

#Call minimisation subject to these values
minout = minimize(fn, x0, args=(A, b), method="SLSQP", bounds=bounds, constraints=cons)
x = minout.x

print(x, x.sum(), fn(x,A,b))
m0 = x

print("")
print("Model estimation:", m0, m0.sum())
print("Check:")
print("Density:", round(np.dot(m0,properties_rho), 2), "-->", round((np.dot(m0,properties_rho)/d[0][0]-1)*100, 2), "% difference")
print("vP:", round(np.dot(m0,properties_vP), 2), "-->", round((np.dot(m0,properties_vP)/d[0][1]-1)*100, 2), "% difference")
print("vS:", round(np.dot(m0,properties_vS), 2), "-->", round((np.dot(m0,properties_vS)/d[0][2]-1)*100, 2), "% difference")
print("GR:", round(np.dot(m0,properties_GR), 2), "-->", round((np.dot(m0,properties_GR)/d[0][3]-1)*100, 2), "% difference")
print("PE:", round(np.dot(m0,properties_PE), 2), "-->", round((np.dot(m0,properties_PE)/d[0][4]-1)*100, 2), "% difference")
print("Reality:")
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
print(d[0])

[0.80 0.00 0.00 0.20] 0.999999817325403 2710.8369823721255

Model estimation: [0.80 0.00 0.00 0.20] 0.999999817325403
Check:
Density: 2318.88 --> 9.38 % difference
vP: 4892.38 --> 74.78 % difference
vS: 3260.72 --> 110.3 % difference
GR: 0.0 --> -100.0 % difference
PE: 1.39 --> -24.8 % difference
Reality:
[2120.00 2799.17 1550.48 51.40 1.85]
